In [ ]:
import json
from os import listdir
from os.path import isfile, join
import re
import string
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# import openai
import time
# from sentence_transformers import SentenceTransformer
# import tiktoken
# from openai.embeddings_utils import get_embedding
from tqdm import tqdm

from sklearn.svm import SVC

import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

import transformers
from transformers import AutoTokenizer, AutoModel, utils
transformers.logging.set_verbosity_error()
#from bertviz import model_view, head_view
utils.logging.set_verbosity_error()  # Suppress standard warnings

# import tensorflow as tf
# from tensorflow import keras
# from sklearn.utils import shuffle


In [ ]:
model_name = "sentence-transformers/bert-base-nli-mean-tokens"
model = AutoModel.from_pretrained(model_name, output_attentions=True)  # Configure model to return attention values
tokenizer = AutoTokenizer.from_pretrained(model_name)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def bert(text):
    sentences = text.split(".")
    embedding_attention_tokens = []

    for i in tqdm(sentences):
        try:
            encoded_input = tokenizer(i, padding=True, truncation=True, return_tensors='pt')
            #print(encoded_input)
            inputs = tokenizer.encode(i, return_tensors='pt')  # Tokenize input text
            outputs = model(inputs)  # Run model
            attention = outputs[-1]  # Retrieve attention from model outputs

            tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 
            sentence_embeddings = mean_pooling(outputs, encoded_input['attention_mask'])
            embedding_attention_tokens.append([i,sentence_embeddings,attention,tokens])
        except:
            embedding_attention_tokens.append([i,-1,-1,-1])
    return embedding_attention_tokens

In [ ]:
cnt=0
temp2=0
for i in train_dataframe.Paper_text.values.tolist():
    cnt+=1
    out = bert(i)
    if cnt==1:
        batches_embedding=out
    else:
        batches_embedding = batches_embedding + out
    
    if cnt==10:
        temp2+=1
        tempd=pd.DataFrame(batches_embedding,columns=['snetence','embeddings','attention','tokens'])
        fname = "batch "+str(temp2)+".csv"
        tempd.to_csv(fname,index=False)
        print("Progress -- {0} ".format(cnt), end="\r")
        batches_embedding=[]
        del tempd
        cnt=0

In [ ]:
# train_bert_embedding = train_dataframe.Paper_text.apply(bert)
# val_bert_embedding = val_dataframe.Paper_text.apply(bert)
# test_bert_embedding = test_dataframe.Paper_text.apply(bert)

In [ ]:
temp1 = train_dataframe.Status.values.tolist()
train_labels=[]
for i in range(0,len(temp1)):
    train_labels.append([temp1[i]]*len(train_bert_embedding[i]))
    
    
temp2 = val_dataframe.Status.values.tolist()
val_labels=[]
for i in range(0,len(temp2)):
    val_labels.append([temp2[i]]*len(val_bert_embedding[i]))
    
temp3 = test_dataframe.Status.values.tolist()
test_labels=[]
for i in range(0,len(temp3)):
    test_labels.append([temp3[i]]*len(test_bert_embedding[i]))
    
train_emebedding_dataframe = pd.DataFrame() 
train_emebedding_dataframe['embeddings']= train_bert_embedding
train_emebedding_dataframe['label']= train_labels


val_emebedding_dataframe = pd.DataFrame() 
val_emebedding_dataframe['embeddings']= val_bert_embedding
val_emebedding_dataframe['label']= val_labels

test_emebedding_dataframe = pd.DataFrame() 
test_emebedding_dataframe['embeddings']= test_bert_embedding
test_emebedding_dataframe['label']= test_labels

In [ ]:
train_emebedding_dataframe.head()

In [ ]:
train_emebedding_dataframe=train_emebedding_dataframe.sample(frac = 1)

In [ ]:
train_emebedding_dataframe[0:10]

In [ ]:
def svm(data,label):
    classifier = SVC(gamma='auto')  
    all_data=[]
    for i in range(0,len(data)):
        for j in range(0,len(data[i])):
            all_data.append([data[i][j],label[i][j]])
    temp= pd.DataFrame(all_data)
    classifier.fit(temp[0].values.tolist(),temp[1].values.tolist())
    return classifier

In [ ]:
svm_model_bert = svm(train_emebedding_dataframe.embeddings.values.tolist(),train_emebedding_dataframe.label.values.tolist())

## Testing accuracy 

In [ ]:
overall_perdiction=[]
sentence_prediction = []
for i in test_emebedding_dataframe.embeddings.values.tolist():
    pred = svm_model_bert.predict(i)
    sentence_prediction.append(pred)
    val = np.bincount(pred).argmax()
    overall_perdiction.append(val)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

mxlenght = 100
padded_sequences = pad_sequences(sentence_prediction, maxlen=mxlenght, padding='post',truncating='pre',value=0.0)

print(padded_sequences)

In [ ]:
from sklearn.linear_model import LogisticRegression
lgclassifier = LogisticRegression()
lgclassifier.fit(X,y)



In [ ]:
test_emebedding_dataframe.head()

In [ ]:
# pred = model.predict(test_emebedding_dataframe.embeddings.values.tolist()[1])
# print(pred)
# np.bincount(pred).argmax()  

In [ ]:
class Classifier(keras.Model):
    def __init__(self, input_dim, num_classes=1, *args, **kwargs):
        super(Classifier, self).__init__(name="Classifier", *args, **kwargs)
        self.DenseL1 = [keras.layers.Dense(512,activation = keras.activations.relu,input_shape=input_dim) for _ in range(2)]
        #self.NoiseLayer = keras.layers.GaussianNoise(stddev=1e-3)
        self.DenseL2 = [keras.layers.Dense(128,activation = keras.activations.relu) for _ in range(2)]
        self.DenseL3 = [keras.layers.Dense(64, activation = keras.activations.relu) for _ in range(2)]
        self.AVLayer = keras.layers.Average()
        self.OutL = keras.layers.Dense(num_classes,activation = tf.nn.softmax)

    def call(self, data, training = False):
        l1_out = [l1(data) for l1 in self.DenseL1]
        #l1_out = [self.NoiseLayer(l1,training=training) for l1 in l1_out]
        l2_out = [(l2(l1)) for l1, l2 in zip(l1_out, self.DenseL2)]
        l3_out = self.AVLayer([l3(l2) for l2, l3 in zip(l2_out, self.DenseL3)])
        l_out = self.OutL(l3_out)
        return l_out


def get_IModel(input_shape, output_shape, decay_steps=1000000, pwr: float = 0.95, starter_learning_rate: float = 1e-2, end_learning_rate: float = 1e-6):
    model = Classifier(input_shape, output_shape)

    learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(starter_learning_rate, decay_steps, end_learning_rate, power=pwr)
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate_fn, clipnorm=1.0, epsilon=1e-09),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=[
        keras.metrics.Recall(),
        keras.metrics.Precision()
    ])
    return model



train_data = []
train_label = []

data,label=train_emebedding_dataframe.embeddings.values.tolist(),train_emebedding_dataframe.label.values.tolist()

for i in range(0,len(data)):
    for j in range(0,len(data[i])):
        train_data.append(data[i][j])
        train_label.append(label[i][j])
        
        
val_data = []
val_label = []

data,label=train_emebedding_dataframe.embeddings.values.tolist(),train_emebedding_dataframe.label.values.tolist()

for i in range(0,len(data)):
    for j in range(0,len(data[i])):
        val_data.append(data[i][j])
        val_label.append(label[i][j])
        
test_data = []
test_label = []

data,label=train_emebedding_dataframe.embeddings.values.tolist(),train_emebedding_dataframe.label.values.tolist()

for i in range(0,len(data)):
    for j in range(0,len(data[i])):
        test_data.append(data[i][j])
        test_label.append(label[i][j])

train_embeddings = pd.DataFrame(train_data)
train_label = pd.DataFrame(train_label)

val_embeddings = pd.DataFrame(val_data)
val_label = pd.DataFrame(val_label)

test_embeddings = pd.DataFrame(test_data)
test_label = pd.DataFrame(test_label)


cnn_model = get_IModel((train_dataframe.shape[1],),1)
#model.load_weights("ModelWei_1")


In [ ]:
out = []
for _ in range(1):
#     tr_data,val_data,tr_label,val_label = train_test_split(d,l,test_size=0.2,random_state=42)
    h = cnn_model.fit(train_embeddings,train_label,validation_data=(val_embeddings,val_label),epochs=10,batch_size=4096)
    out.append(h)


In [ ]:
overall_pred = []
cnn_pred=[]
for i in test_emebedding_dataframe.embeddings:
    temp_pred = cnn_model.predict(i)
    temp=[]
    for j in temp_pred:
        temp.append(j[0])
    cnn_pred.append(temp)
    overall_pred.append(np.bincount(temp).argmax())
    

In [ ]:
overall_pred

In [ ]:
cnn_model.save_weights("Modelcnnbert")

In [ ]:
import pickle

# save
with open('svcbert.pkl','wb') as f:
    pickle.dump(svm_model_bert,f)


In [ ]:
train_emebedding_dataframe.to_csv('bert_embedding_train',index=False)
val_emebedding_dataframe.to_csv('bert_embedding_val',index=False)
test_emebedding_dataframe.to_csv('bert_embedding_test',index=False)

## Testing samples codes

In [ ]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view, head_view
utils.logging.set_verbosity_error()  # Suppress standard warnings
model_name = "sentence-transformers/bert-base-nli-mean-tokens"
model = AutoModel.from_pretrained(model_name, output_attentions=True)  # Configure model to return attention values
tokenizer = AutoTokenizer.from_pretrained(model_name)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def bert(text):
    sentences = text.split(".")
    embedding_attention_tokens = []

    for i in sentences:
        encoded_input = tokenizer(i, padding=True, truncation=True, return_tensors='pt')
        print(encoded_input)
        inputs = tokenizer.encode(i, return_tensors='pt')  # Tokenize input text
        #print(inputs)
        outputs = model(inputs)  # Run model
        attention = outputs[-1]  # Retrieve attention from model outputs

        tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 
        sentence_embeddings = mean_pooling(outputs, encoded_input['attention_mask'])
        embedding_attention_tokens.append([sentence_embeddings,attention,tokens,encoded_input])
    return embedding_attention_tokens

In [ ]:
temp = bert("I am kashyap. I am powerfull")

In [ ]:
model_view(temp[0][1], temp[0][2])  # Display model view
head_view(temp[0][1], temp[0][2])

In [ ]:
import captum
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization, IntegratedGradients

In [ ]:
lig = LayerIntegratedGradients(model, model.embeddings)
token_reference = TokenReferenceBase(reference_token_idx=tokenizer.pad_token_id)

In [ ]:
temp[0][3]['input_ids']